# Configurações

## Importações

In [21]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib

## Definição do Dataframe

In [22]:
df = pd.read_csv("../datasets/features_normalized_with_decades.csv")

# Modelagem

In [23]:
X_train, X_test = train_test_split(df, test_size=0.3, random_state=42)

model = NearestNeighbors(n_neighbors=100, metric="euclidean", algorithm="kd_tree")
model.fit(X_train)

print(f"Modelo treinado: {len(X_train)} amostras")
print(f"Teste: {len(X_test)} amostras")

Modelo treinado: 118934 amostras
Teste: 50973 amostras


# Avaliação do modelo

In [24]:
distances, indices = model.kneighbors(X_test.iloc[:5])

print("=" * 90)
print("📊 AVALIAÇÃO DE AMOSTRAS")
print("=" * 90)

for i in range(5):
    print(f"\n🎵 Amostra {i}:")
    print(
        f"   Distâncias dos 5 vizinhos mais próximos: {np.round(distances[i][:5], 4)}"
    )
    print(f"   Índices: {indices[i][:5]}")

print("\n" + "=" * 90)
print("📈 MÉTRICAS DO MODELO")
print("=" * 90)

distances_all, _ = model.kneighbors(X_test)

print(f"\n✅ Distância média euclidiana: {distances_all.mean():.4f}")
print(f"✅ Distância mínima: {distances_all.min():.4f}")
print(f"✅ Distância máxima: {distances_all.max():.4f}")
print(f"✅ Desvio padrão: {distances_all.std():.4f}")
print(f"\n✅ Amostras de teste: {len(X_test)}")
print("✅ Vizinhos encontrados por amostra: 20")
print(f"✅ Total de recomendações possíveis: {len(X_test) * 20}")

📊 AVALIAÇÃO DE AMOSTRAS

🎵 Amostra 0:
   Distâncias dos 5 vizinhos mais próximos: [0.1049 0.1212 0.1397 0.1616 0.1971]
   Índices: [ 82603 105944 117978 101321  82282]

🎵 Amostra 1:
   Distâncias dos 5 vizinhos mais próximos: [0.1898 0.2448 0.2503 0.2518 0.2596]
   Índices: [80494 57900 39396 88828 13360]

🎵 Amostra 2:
   Distâncias dos 5 vizinhos mais próximos: [0.2257 0.2986 0.3187 0.3428 0.3456]
   Índices: [38405 24485 99111 30134 50325]

🎵 Amostra 3:
   Distâncias dos 5 vizinhos mais próximos: [0.1393 0.1421 0.156  0.1636 0.1663]
   Índices: [44466 59319 70010 88130 72590]

🎵 Amostra 4:
   Distâncias dos 5 vizinhos mais próximos: [0.1768 0.2046 0.2153 0.2277 0.2312]
   Índices: [54353 28463 82308 71324 38513]

📈 MÉTRICAS DO MODELO

✅ Distância média euclidiana: 0.4887
✅ Distância mínima: 0.0000
✅ Distância máxima: 2.5865
✅ Desvio padrão: 0.2314

✅ Amostras de teste: 50973
✅ Vizinhos encontrados por amostra: 20
✅ Total de recomendações possíveis: 1019460


# Gerar o modelo

In [26]:
os.makedirs("../models", exist_ok=True)

joblib.dump(model, "../models/music_recommender_model.joblib")
joblib.dump(df.columns.tolist(), "../models/music_model_features.pkl")

print("Modelo salvo com sucesso")

Modelo salvo com sucesso
